<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block; font-weight:bold; font-size:40px;">
 SQL Analysis Queries by python
</p>

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;">
1. Sales Analysis<br>
- Top 10 best-selling products<br>
- Top 5 customers by spending<br>
- Revenue per store<br>
- Revenue per category<br>
- Monthly sales trend
</p>

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;">
2. Inventory Analysis<br>
- Products with low stock<br>
- Stores with the highest inventory levels
</p>

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;">
3. Staff Performance<br>
- Number of orders handled by each staff member<br>
- Best-performing staff member by total sales
</p>

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;">
4. Customer Insights<br>
- Customers with no orders<br>
- Average spending per customer
</p>

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# ------------------------------
server = "DESKTOP-05DRQII"
database = "RetailDB"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes")

# ------------------------------

queries = {
    "top_products": """
        SELECT TOP 10 
            p.product_name,
            SUM(oi.quantity) AS total_quantity_sold,
            SUM(oi.total_price) AS total_sales
        FROM OrderItems oi
        JOIN Products p ON oi.product_id = p.product_id
        GROUP BY p.product_name
        ORDER BY total_sales DESC;
    """,
    
    "top_customers": """
        SELECT TOP 5
            c.full_name,
            SUM(oi.total_price) AS total_spent,
            COUNT(DISTINCT o.order_id) AS total_orders
        FROM Orders o
        JOIN Customers c ON o.customer_id = c.customer_id
        JOIN OrderItems oi ON o.order_id = oi.order_id
        GROUP BY c.full_name
        ORDER BY total_spent DESC;
    """,
    
    "revenue_per_store": """
        SELECT
            s.store_name,
            SUM(oi.total_price) AS total_revenue
        FROM Orders o
        JOIN Stores s ON o.store_id = s.store_id
        JOIN OrderItems oi ON o.order_id = oi.order_id
        GROUP BY s.store_name
        ORDER BY total_revenue DESC;
    """,

    "revenue_per_category": """
        SELECT
            c.category_name,
            SUM(oi.total_price) AS total_revenue
        FROM OrderItems oi
        JOIN Products p ON oi.product_id = p.product_id
        JOIN Categories c ON p.category_id = c.category_id
        GROUP BY c.category_name
        ORDER BY total_revenue DESC;
    """,

    "monthly_sales": """
        SELECT
            FORMAT(CONVERT(DATE, o.order_date), 'yyyy-MM') AS month,
            SUM(oi.total_price) AS total_sales
        FROM Orders o
        JOIN OrderItems oi ON o.order_id = oi.order_id
        GROUP BY FORMAT(CONVERT(DATE, o.order_date), 'yyyy-MM')
        ORDER BY month;
    """,

    "low_stock": """
        SELECT
            p.product_name,
            SUM(s.quantity) AS total_stock
        FROM Stocks s
        JOIN Products p ON s.product_id = p.product_id
        GROUP BY p.product_name
        HAVING SUM(s.quantity) < 50
        ORDER BY total_stock ASC;
    """,

    "highest_inventory": """
        SELECT
            st.store_name,
            SUM(s.quantity) AS total_inventory
        FROM Stocks s
        JOIN Stores st ON s.store_id = st.store_id
        GROUP BY st.store_name
        ORDER BY total_inventory DESC;
    """,

    "staff_performance": """
        SELECT
            st.first_name + ' ' + st.last_name AS staff_name,
            COUNT(o.order_id) AS total_orders,
            SUM(oi.total_price) AS total_sales
        FROM Orders o
        JOIN Staffs st ON o.staff_id = st.staff_id
        JOIN OrderItems oi ON o.order_id = oi.order_id
        GROUP BY st.first_name, st.last_name
        ORDER BY total_sales DESC;
    """,

    "customers_no_orders": """
        SELECT
            c.full_name
        FROM Customers c
        LEFT JOIN Orders o ON c.customer_id = o.customer_id
        WHERE o.order_id IS NULL;
    """,

    "average_spending": """
        SELECT
            sub.full_name,
            AVG(sub.customer_total) AS avg_spending,
            COUNT(sub.customer_total) AS total_orders
        FROM (
            SELECT
                c.customer_id,
                c.full_name,
                SUM(oi.total_price) AS customer_total
            FROM Customers c
            JOIN Orders o ON c.customer_id = o.customer_id
            JOIN OrderItems oi ON o.order_id = oi.order_id
            GROUP BY c.customer_id, c.full_name
        ) AS sub
        GROUP BY sub.full_name
        ORDER BY avg_spending DESC;
    """
}

# ------------------------------

for name, query in queries.items():
    df = pd.read_sql(query, engine)
    df.to_csv(f"{name}.csv", index=False)
    print(f"{name}.csv saved successfully!")


top_products.csv saved successfully!
top_customers.csv saved successfully!
revenue_per_store.csv saved successfully!
revenue_per_category.csv saved successfully!
monthly_sales.csv saved successfully!
low_stock.csv saved successfully!
highest_inventory.csv saved successfully!
staff_performance.csv saved successfully!
customers_no_orders.csv saved successfully!
average_spending.csv saved successfully!


________________